# Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
from pathlib import Path

from PIL import Image
from google import genai

import tt

LOG = logging.getLogger(__name__)
tt.logging_init()

# logging.root.manager.loggerDict
logging.getLogger("google_genai").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

In [ ]:
CATEGORIES = [
    "chicken",
    "cow",
    "creeper",
    "enderman",
    "pig",
    "sheep",
    "skeleton",
    "spider",
    "zombie",
]
CATEGORIES.sort()
CATEGORIES_MINECRAFT = [f"minecraft {x}" for x in CATEGORIES]

flash = "gemini-2.5-flash"
flash_lite = "gemini-2.5-flash-lite"
prompt_short = f"""Detect {", ".join(CATEGORIES)}. Return AT MOST 20 detections."""

mobs_path = Path.home() / "src/data/mobs2"
images_path = mobs_path / "640"

QCFG = tt.GeminiQueryConfig(prompt=prompt_short, categories=CATEGORIES)

# File upload

In [ ]:
GF = tt.GeminiFileAPI()
# GF.clear()
# GF.upload_dir(images_path, slice_obj=slice(None))
GF.sync()
print(f"n_files={len(GF.gfiles)}")
print(GF.gfiles)

# 

# Detect

In [ ]:
# This image causes gemini-2.5-flash-lite to detect the pig repeatedly.
image_break = "/Users/joe/src/data/mobs1/640/cda62cfe-2024-10-20_22.22.31.png"
image640 = Image.open(image_break)
# display(image640)
bbox = tt.gemini_gen_bboxes(image640, QCFG)
print(bbox)
display(tt.plot_bb(image640, bbox, CATEGORIES))

In [ ]:
# This causes gemini to identify a bunch of ice blocks
image_break = "/Users/joe/src/data/mobs2/640/frame_10835.png"
image640 = Image.open(image_break)
# display(image640)
bbox = tt.gemini_detect_gfile(image640, QCFG)
print(bbox)
display(tt.plot_bb(image640, bbox, CATEGORIES))

In [ ]:
# Test single query
image640 = Image.open(sorted(images_path.glob("*.png"))[5])
bbox = tt.gemini_detect_single(image640, QCFG)
print(bbox)
display(tt.plot_bb(image640, bbox, CATEGORIES))

In [ ]:
# Display results of one query at a time
def detect_single(gfile: genai.types.File):
    return tt.gemini_detect_gfile(gfile, QCFG)
tt.InferViewer[genai.types.File](detect_single, GF.gfiles, QCFG.categories).show_widget()

In [ ]:
# Run all queries first, then display the results
# GFILES10 = GF.gfiles[:10]
# results = tt.gemini_detect_gfile_multi(GFILES10, QCFG)
results = tt.gemini_detect_gfile_multi(GF.gfiles, QCFG)
tt.InferViewer[tt.ImageResult](lambda x: x, results, QCFG.categories).show_widget()

In [ ]:
dset = tt.Dataset(categories=QCFG.categories, images=results)
dset.save(mobs_path / "gemini_results.json")